In [17]:
from flask import Flask, jsonify, request, render_template
from flask_cors import CORS
from joblib import load
import numpy as np
from datetime import datetime
import pandas as pd
import logging

app = Flask(__name__)
CORS(app)

# Configure logging
logging.basicConfig(level=logging.INFO)

try:
    # Load the scaler and the model
    scaler = load('scaler.joblib')
    model = load('best_XGB_model.joblib')
except Exception as e:
    logging.error(f"Error loading model or scaler: {str(e)}")
    raise

market_segment_map = {
    'Offline': 1,
    'Online': 0
}

# Define expected columns based on preprocessing
expected_columns = ['lead time', 'average price', 'special requests', 'day', 'month', 
                    'number of weekend nights', 'number of week nights', 'Online', 
                    'number of adults', 'year', 'Offline']

@app.route('/')
def home():
    return jsonify({"message": "Hello, World!"})

@app.route('/predict', methods=['POST', 'OPTIONS'])
def predict():
    if request.method == 'OPTIONS':
        return _build_cors_prelight_response()

    try:
        data = request.get_json()

        # Validate input data
        if not data:
            raise ValueError("No data provided")

        required_fields = ['numAdults', 'numWeekendNights', 'numWeekNights', 'leadTime',
                           'averagePrice', 'specialRequests', 'reservationDate', 'marketSegment']

        for field in required_fields:
            if field not in data:
                raise ValueError(f"Missing field: {field}")

        # Convert categorical variables to numerical values
        market_segment = market_segment_map.get(data['marketSegment'], 0)

        # Extract date components
        reservation_date = datetime.fromisoformat(data['reservationDate'])
        day = reservation_date.day
        month = reservation_date.month
        year = reservation_date.year

        # Prepare the features for prediction
        features = {
            'number of adults': int(data['numAdults']),
            'number of weekend nights': int(data['numWeekendNights']),
            'number of week nights': int(data['numWeekNights']),
            'lead time': int(data['leadTime']),
            'average price': float(data['averagePrice']),  # Ensure this matches model expectations
            'special requests': int(data['specialRequests']),
            'day': day,
            'month': month,
            'year': year,
            'Offline': 1 if data['marketSegment'] == 'Offline' else 0,
            'Online': 1 if data['marketSegment'] == 'Online' else 0
        }

        # Create a DataFrame from features
        features_df = pd.DataFrame([features])

        # Reorder columns to match expected order
        features_df = features_df[expected_columns]

        # Scale the features
        features_scaled = scaler.transform(features_df)

        # Make prediction
        prediction = model.predict(features_scaled)[0]
        prediction_result = "Canceled" if prediction == 1 else "Not Canceled"

        return jsonify({"prediction": prediction_result})

    except ValueError as e:
        logging.error(f"Value error: {str(e)}")
        return jsonify({"error": str(e)}), 400

    except Exception as e:
        logging.error(f"Prediction error: {str(e)}")
        return jsonify({"error": "An error occurred during prediction"}), 500


def _build_cors_prelight_response():
    response = jsonify()
    response.headers.add("Access-Control-Allow-Origin", "*")
    response.headers.add("Access-Control-Allow-Headers", "*")
    response.headers.add("Access-Control-Allow-Methods", "*")
    return response, 200


if __name__ == "__main__":
    try:
        app.run(debug=True)
    except Exception as e:
        import traceback
        traceback.print_exc()
        raise


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1